<a href="https://colab.research.google.com/github/neftali-lopez-elizondo/Decimal-Microtonal-Scale/blob/main/Reproductor_de_MDMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------------------------------------------------------------
# MDMI (Musical Decimal Microtonal Interface)
# Copyright (c) 2024, [Tu Nombre o Compañía]. Todos los derechos reservados.
#
# Este software está distribuido bajo la Licencia MIT.
# Para más información, visita: https://github.com/[tu-usuario]/MDMI
#
# ----------------------------------------------------------------------------
# Este archivo reproduce un archivo MDMI (JSON) utilizando una escala
# de 10 tonos por octava.
# ----------------------------------------------------------------------------

import json
import numpy as np
import time
import tkinter as tk
from tkinter import filedialog
import pygame

# Inicializa la librería de audio de pygame
pygame.mixer.init(frequency=44100, size=-16, channels=1)

# Constantes para la afinación
LA_FRECUENCIA_HZ = 440.0
LA_TONO_DECIMAL = 84

# --------------------------
# Funciones de utilidad
# --------------------------
def get_pitch_from_decimal_tone(decimal_tone):
    """
    Convierte un tono decimal a frecuencia en Hz (10 tonos por octava).
    """
    pitch_in_octaves = (decimal_tone - LA_TONO_DECIMAL) / 10.0
    frequency_hz = LA_FRECUENCIA_HZ * (2.0 ** pitch_in_octaves)
    return frequency_hz

def generate_sine_wave(frequency, duration, sample_rate=44100):
    """
    Genera una onda sinusoidal para una frecuencia y duración dadas.
    """
    num_samples = int(duration * sample_rate)
    t = np.linspace(0, duration, num_samples, False)
    # Genera una onda sinusoidal pura
    audio = np.sin(2 * np.pi * frequency * t)
    # Normaliza a 16-bit
    audio *= 32767 / np.max(np.abs(audio)) * 0.5  # Reduce el volumen a la mitad
    return audio.astype(np.int16)

def play_mdmi_file(file_path):
    """
    Lee un archivo MDMI y reproduce la música.
    """
    print(f"🎵 Reproduciendo '{file_path}'...")
    with open(file_path, "r") as f:
        mdmi_data = json.load(f)

    frequency_deltas = []
    last_frequency = None

    for track in mdmi_data["tracks"]:
        print(f"   Pista: {track['track_name']}")
        last_time = 0
        for note in track["notes"]:
            wait_time = (note["time"] - last_time) * (mdmi_data["metadata"]["tempo"] / 1000000.0) / \
                         (mdmi_data["metadata"]["ticks_per_beat"])

            if wait_time > 0:
                time.sleep(wait_time)

            frequency = get_pitch_from_decimal_tone(note["tone"])
            duration = note["duration"]

            # --- Métricas y Análisis ---
            if last_frequency is not None:
                delta_hz = frequency - last_frequency
                frequency_deltas.append(round(delta_hz, 2))
                print(f"       -> Frecuencia: {frequency:.2f} Hz, Delta: {delta_hz:.2f} Hz")
            else:
                print(f"       -> Frecuencia: {frequency:.2f} Hz")

            last_frequency = frequency

            wave = generate_sine_wave(frequency, duration)
            sound = pygame.sndarray.make_sound(wave)
            sound.play()

            last_time = note["time"]

    print("✅ Reproducción completa.")
    print("--- Análisis de Frecuencias ---")
    print(f"Vector de Deltas (Hz): {frequency_deltas}")
    if frequency_deltas:
        print(f"Promedio de Delta: {np.mean(frequency_deltas):.2f} Hz")
        print(f"Desviación Estándar de Delta: {np.std(frequency_deltas):.2f} Hz")

# --------------------------
# Interfaz de usuario con Tkinter
# --------------------------
def select_mdmi_file():
    """Muestra un diálogo para seleccionar un archivo MDMI y lo reproduce."""
    root = tk.Tk()
    root.withdraw()  # Oculta la ventana principal

    file_path = filedialog.askopenfilename(
        title="Selecciona un archivo MDMI para reproducir",
        filetypes=[("MDMI files", "*.mdmi")]
    )

    if file_path:
        play_mdmi_file(file_path)
    else:
        print("Operación cancelada. No se seleccionó ningún archivo.")

if __name__ == "__main__":
    select_mdmi_file()